In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# 1) Gather all file paths
file_list = []
for root, _, files in os.walk('/kaggle/input'):
    for f in files:
        file_list.append({'file_path': os.path.join(root, f)})

# 2) Load into a DataFrame
test_df = pd.DataFrame(file_list)




/kaggle/input/task3_test_without_label.jsonl
/kaggle/input/._task3_test_without_label.jsonl
/kaggle/input/images/twitter_images/x.com_antipccomics/x.com_antipccomics/GX19Sw1XUAAZnIN.jpg
/kaggle/input/images/twitter_images/x.com_antipccomics/x.com_antipccomics/GenD60kWcAEKg1G.jpg
/kaggle/input/images/twitter_images/x.com_antipccomics/x.com_antipccomics/Giov3CdW8AECmMs.jpg
/kaggle/input/images/twitter_images/x.com_memeskawmia626/x.com_memeskawmia626/GlNboNwXMAIsgfQ.jpg
/kaggle/input/images/twitter_images/x.com_memeskawmia626/x.com_memeskawmia626/GjBxAMEXAAA0Ibw.jpg
/kaggle/input/images/twitter_images/x.com_memeskawmia626/x.com_memeskawmia626/GhbLIRJXYAAkwiz.jpg
/kaggle/input/images/twitter_images/x.com_memeskawmia626/x.com_memeskawmia626/Gh5sGlbWIAAg26w.jpg
/kaggle/input/images/twitter_images/x.com_memeskawmia626/x.com_memeskawmia626/GlxQ2S9WQAAiQqG.jpg
/kaggle/input/images/twitter_images/x.com_memeskawmia626/x.com_memeskawmia626/Gh-hnasXkAA894b.jpg
/kaggle/input/images/twitter_images/x.

In [2]:
#cell-1
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [3]:
#cell-2
%matplotlib inline

import json
import logging
from pathlib import Path
import random
import tarfile
import tempfile
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torchvision

In [4]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
splits = {'train': 'data/train-00000-of-00001-d9ac21f8cab19f0c.parquet', 'test': 'data/test-00000-of-00001-3d9abbec7a460a82.parquet', 'dev': 'data/dev-00000-of-00001-1c8a7be6beca8257.parquet'}
df = pd.read_parquet("hf://datasets/QCRI/Prop2Hate-Meme/" + splits["train"])

In [5]:
# Load the validation data and assign to val_samples_frame
val_samples_frame = pd.read_parquet("hf://datasets/QCRI/Prop2Hate-Meme/" + splits["dev"])

# Assign the already loaded training data (df) to train_samples_frame
train_samples_frame = df

print("Train DataFrame Head:")
display(train_samples_frame.head())
print("\nValidation DataFrame Head:")
display(val_samples_frame.head())

Train DataFrame Head:


,id,text,image,img_path,prop_label,hate_label,hate_fine_grained_label
0,data/arabic_memes_fb_insta_pinterest/Pinterest...,زوجة ماكرون تصرح أن الحجاب يرعب ويخيف الأطفال....,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,1,0,0
1,data/arabic_memes_fb_insta_pinterest/Pinterest...,#ده حال الادمن لمابنت تشتكيله من عضو دايقها ==ئ=,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,0,1
2,data/arabic_memes_fb_insta_pinterest/Pinterest...,سما الصدي : عاوزة آتجوز محمد صلاح شهر واحد وهي...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,0,1
3,data/arabic_memes_fb_insta_pinterest/Instagram...,اما بابا يصحي الفجر ويلاقينا انا وماما واختي ل...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,0,1
4,data/arabic_memes_fb_insta_pinterest/Facebook/...,بم معربه معزن ولرانبه\nطب والله العظيم لاحلق ش...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Faceboo...,0,0,1



Validation DataFrame Head:


,id,text,image,img_path,prop_label,hate_label,hate_fine_grained_label
0,data/arabic_memes_fb_insta_pinterest/Pinterest...,للطب مذاق خاص لا يشعر بلذته إلا من تذوقه..,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,0,1
1,data/arabic_memes_fb_insta_pinterest/Instagram...,No one:\nقطة السلم :\nبصى انا رحت اتجوزت وخلفت...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,0,1
2,data/fb_memes/ArabianMemez/42303742_4678711737...,عندما تكرس حياتك في صناعة الميمز من أجل إضحاك ...,{'bytes': b'\xff\xd8\xff\xe2\x0b\xf8ICC_PROFIL...,./data/fb_memes/ArabianMemez/42303742_46787117...,1,0,0
3,data/arabic_memes_fb_insta_pinterest/Pinterest...,المراقب \nإتأكد من إسمك ورقم جلوسك!!\n\nهو أنا...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,0,1
4,data/arabic_memes_fb_insta_pinterest/Pinterest...,يابني مش هتبطل كل اللى يكلمك تاخده ع قد عقله ؟...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,0,1


In [6]:
#cell-5
print(train_samples_frame.columns)


Index(['id', 'text', 'image', 'img_path', 'prop_label', 'hate_label',
       'hate_fine_grained_label'],
      dtype='object')


In [7]:
train_samples_frame.hate_label.value_counts()

hate_label
0    1930
1     213
Name: count, dtype: int64

In [8]:
#cell-6
train_samples_frame['length'] = train_samples_frame['text'].apply(len)
# Define the number of bins
num_bins = 3

# Create bins automatically
train_samples_frame['length_bins'] = pd.cut(train_samples_frame['length'], bins=num_bins)

In [9]:
#cell-7
max_length = train_samples_frame['length'].max()
print("Maximum Length:", max_length)
median_length = train_samples_frame['length'].median()
print("Median Length:", median_length)

Maximum Length: 475
Median Length: 66.0


In [10]:
train_samples_frame

,id,text,image,img_path,prop_label,hate_label,hate_fine_grained_label,length,length_bins
0,data/arabic_memes_fb_insta_pinterest/Pinterest...,زوجة ماكرون تصرح أن الحجاب يرعب ويخيف الأطفال....,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,1,0,0,50,"(5.531, 162.333]"
1,data/arabic_memes_fb_insta_pinterest/Pinterest...,#ده حال الادمن لمابنت تشتكيله من عضو دايقها ==ئ=,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,0,1,48,"(5.531, 162.333]"
2,data/arabic_memes_fb_insta_pinterest/Pinterest...,سما الصدي : عاوزة آتجوز محمد صلاح شهر واحد وهي...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,0,1,166,"(162.333, 318.667]"
3,data/arabic_memes_fb_insta_pinterest/Instagram...,اما بابا يصحي الفجر ويلاقينا انا وماما واختي ل...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,0,1,55,"(5.531, 162.333]"
4,data/arabic_memes_fb_insta_pinterest/Facebook/...,بم معربه معزن ولرانبه\nطب والله العظيم لاحلق ش...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Faceboo...,0,0,1,83,"(5.531, 162.333]"
...,...,...,...,...,...,...,...,...,...
2138,data/arabic_memes_fb_insta_pinterest/Instagram...,: Fact\nانا بقيت نسخه سخيفه وكئيبة\nمن إنسان ك...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,0,1,62,"(5.531, 162.333]"
2139,data/arabic_memes_fb_insta_pinterest/Pinterest...,الرجل هم المرهم والمرهم لاطيب للعيش بلا هم وا...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,1,7,120,"(5.531, 162.333]"
2140,data/arabic_memes_fb_insta_pinterest/Instagram...,لما أطلع غلطانه وماعنديش حاجه ادافع عن نفسي في...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,0,1,63,"(5.531, 162.333]"
2141,data/arabic_memes_fb_insta_pinterest/Instagram...,-أهلي :- متتحركيش من مكانك في الفرح وتفضلي لاز...,{'bytes': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,0,1,133,"(5.531, 162.333]"


In [11]:
train_samples_frame = train_samples_frame[
    (train_samples_frame['hate_label'] == 0) & (train_samples_frame['length'] < 62) |
    (train_samples_frame['hate_label'] == 1)
]


In [12]:
print("Rows available:", len(train_samples_frame))


Rows available: 1079


In [13]:
train_samples_frame.hate_label.value_counts()

hate_label
0    866
1    213
Name: count, dtype: int64

In [14]:
#cell-8
train_texts = list(train_samples_frame['text'])
val_texts = list(val_samples_frame['text'])
train_labels = list(train_samples_frame['hate_label'])
val_labels = list(val_samples_frame['hate_label'])

In [15]:
# cell-9 (updated): Process image data directly from the dataframe

from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
import io
from PIL import Image

# ─── Build training images array from dataframe 'image' column────────────────────
train_images = []
for index, row in train_samples_frame.iterrows():
    image_data = row['image']['bytes'] # Access the 'bytes' key within the dictionary in the 'image' column
    img = Image.open(io.BytesIO(image_data))
    img = img.resize((128, 128)) # Resize the image
    arr = img_to_array(img) / 255.0
    train_images.append(arr)

train_images = np.stack(train_images)

# ─── Build validation images array from dataframe 'image' column──────────────────
val_images = []
for index, row in val_samples_frame.iterrows():
    image_data = row['image']['bytes'] # Access the 'bytes' key within the dictionary in the 'image' column
    img = Image.open(io.BytesIO(image_data))
    img = img.resize((128, 128)) # Resize the image
    arr = img_to_array(img) / 255.0
    val_images.append(arr)

val_images = np.stack(val_images)

# ─── Sanity check ─────────────────────────────────────────────────────────────
print(f"Loaded {train_images.shape[0]} training images of shape {train_images.shape[1:]}")
print(f"Loaded {val_images.shape[0]} validation images of shape {val_images.shape[1:]}")

2025-07-27 08:31:34.193424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753605094.407502      71 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753605094.474629      71 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loaded 1079 training images of shape (128, 128, 3)
Loaded 312 validation images of shape (128, 128, 3)


In [16]:
#cell-10
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D,Flatten
from transformers import TFBertModel, BertTokenizer
import numpy as np

# Load pre-trained BERT model and tokenizer
bert_model = TFBertModel.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="tf", padding=True, truncation=True, max_length=128)
    outputs = bert_model(inputs)
    outputs = outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token embedding
    return outputs

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

I0000 00:00:1753605128.842926      71 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertFo

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [17]:
# cell-11
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer
import numpy as np



# ─── Batched BERT embedding function ─────────────────────────────
def get_bert_embeddings(texts, batch_size=32):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size]
        inputs = tokenizer(
            batch_texts,
            return_tensors="tf",
            padding=True,
            truncation=True,
            max_length=128
        )
        outputs = bert_model(inputs)
        cls_embeddings = outputs.last_hidden_state[:, 0, :]  # Use [CLS] token
        all_embeddings.append(cls_embeddings)
    return tf.concat(all_embeddings, axis=0)

# ─── Generate BERT embeddings for train and validation ───────────
train_text_embeddings = get_bert_embeddings(train_texts, batch_size=32)
val_text_embeddings   = get_bert_embeddings(val_texts, batch_size=32)

# ─── Convert to NumPy arrays ─────────────────────────────────────
train_text_embeddings = np.array(train_text_embeddings)
val_text_embeddings   = np.array(val_text_embeddings)

# ─── Sanity check ────────────────────────────────────────────────
print("Train text embedding shape:", train_text_embeddings.shape)
print("Validation text embedding shape:", val_text_embeddings.shape)


Train text embedding shape: (1079, 768)
Validation text embedding shape: (312, 768)


In [18]:
#cell-12
train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [19]:
#cell-13
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling2D,Flatten, Dropout

# Load pre-trained InceptionResNetV2 model, exclude top layers
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(128 , 128, 3))

# Custom model combining InceptionResNetV2 and BERT embeddings
image_input = base_model.input
x = base_model(image_input)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)

text_input = Input((768,))
text_input = Flatten()(text_input)

combined = Concatenate()([x, text_input])

x_last = Dense(1024, activation = 'relu')(combined)
x_last = Dense(512, activation = 'relu')(x_last)
x_last = Dropout(0.5)(x_last)
x_last = Dense(256, activation = 'relu')(x_last)
x_last = Dense(128, activation = 'relu')(x_last)
x_last = Dropout(0.5)(x_last)
predictions = Dense(2, activation='softmax')(x_last)


# Define the final model
model = Model(inputs=[image_input, text_input], outputs=predictions)

# Compile the model
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [20]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ inception_resnet_v2 │ (None, 2, 2,      │ 54,336,736 │ input_layer[1][0] │
│ (Functional)        │ 1536)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1536)      │          0 │ inception_resnet… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 1536)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ keras_tensor_784CL… │ (None, 768)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 2304)      │          0 │ flatten[1][0],    │
│ (Concatenate)       │                   │            │ keras_tensor_784… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 1024)      │  2,360,320 │ concatenate[1][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 512)       │    524,800 │ dense[1][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ dense_1[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 256)       │    131,328 │ dropout[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 128)       │     32,896 │ dense_2[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_3[1][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 2)         │        258 │ dropout_1[1][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,386,338 (218.91 MB)

 Trainable params: 57,325,794 (218.68 MB)

 Non-trainable params: 60,544 (236.50 KB)

In [21]:
#cell-14
# Train the model
model.fit([train_images, train_text_embeddings], train_labels, epochs=16, batch_size=32,validation_data=([val_images, val_text_embeddings], val_labels))

Epoch 1/16


I0000 00:00:1753605286.800038     151 service.cc:148] XLA service 0x7fbfb451a200 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1753605286.801038     151 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1753605300.335990     151 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1753605345.116323     151 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


34/34 ━━━━━━━━━━━━━━━━━━━━ 270s 2s/step - accuracy: 0.7446 - loss: 0.6708 - val_accuracy: 0.9006 - val_loss: 19.8292
Epoch 2/16
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 170ms/step - accuracy: 0.7896 - loss: 0.5345 - val_accuracy: 0.9006 - val_loss: 201.1270
Epoch 3/16
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - accuracy: 0.8036 - loss: 0.4081 - val_accuracy: 0.8974 - val_loss: 10.8590
Epoch 4/16
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 167ms/step - accuracy: 0.8346 - loss: 0.3956 - val_accuracy: 0.9006 - val_loss: 575.2642
Epoch 5/16
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - accuracy: 0.9094 - loss: 0.2409 - val_accuracy: 0.7724 - val_loss: 9.7194
Epoch 6/16
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 169ms/step - accuracy: 0.9269 - loss: 0.2217 - val_accuracy: 0.8622 - val_loss: 93.2499
Epoch 7/16
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - accuracy: 0.9439 - loss: 0.1678 - val_accuracy: 0.7885 - val_loss: 6.5387
Epoch 8/16
34/34 ━━━━━━━━━━━━━━━━━━━━ 6s 168ms/step - accuracy: 0.9749 - loss: 0.1254 - val_accuracy: 0.7051 

In [22]:
#cell-15
predictions = model.predict([val_images, val_text_embeddings])

10/10 ━━━━━━━━━━━━━━━━━━━━ 28s 2s/step


In [23]:
#cell-16
predictions = np.argmax(predictions, axis=1)
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, predictions)

0.8493589743589743

In [24]:
#cell-17
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(val_labels, predictions, target_names = ['non_hateful','hateful']))
cm = confusion_matrix(val_labels, predictions)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=['non_hateful','hateful'], yticklabels= ['non_hateful','hateful'])
plt.title('Confusion Matrix')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

              precision    recall  f1-score   support

 non_hateful       0.96      0.87      0.91       281
     hateful       0.36      0.65      0.46        31

    accuracy                           0.85       312
   macro avg       0.66      0.76      0.69       312
weighted avg       0.90      0.85      0.87       312



NameError: name 'sns' is not defined

<Figure size 1000x800 with 0 Axes>

In [25]:
import os
import json
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 1️⃣ Correct path to JSONL
jsonl_path = "/kaggle/input/task3_test_without_label.jsonl"

# 2️⃣ Load JSONL into DataFrame
records = []
with open(jsonl_path, "r") as f:
    for line in f:
        if line.strip():
            records.append(json.loads(line))

test_df = pd.DataFrame(records)
print("Columns in DataFrame:", test_df.columns)
print(test_df.head())

# 3️⃣ Point BASE_INPUT at your dataset root
BASE_INPUT = "/kaggle/input"

# 4️⃣ Resolve image paths
def resolve_image_path(p: str) -> str:
    """
    Turn JSONL’s "./data/…" paths into the real disk locations under BASE_INPUT.
    """
    if p.startswith("./"):
        p = p[2:]
    if "data/archive/" in p:
        rel = p.split("data/archive/")[-1]
        return os.path.join(BASE_INPUT, "archive", rel)
    else:
        rel = p.split("data/images/")[-1]
        return os.path.join(BASE_INPUT, "images", rel)

# Use the correct column from test_df (replace 'img_path' if the column name differs)
if 'img_path' not in test_df.columns:
    # Try to auto-detect the column that contains image paths
    path_col = [c for c in test_df.columns if test_df[c].astype(str).str.contains("data/").any()]
    if path_col:
        path_col = path_col[0]
    else:
        raise KeyError("Could not find an image path column in the JSONL file.")
else:
    path_col = 'img_path'

test_df["image_file"] = test_df[path_col].apply(resolve_image_path)

# 5️⃣ Quick sanity check
missing = test_df["image_file"].apply(lambda p: not os.path.exists(p)).sum()
print(f"⚠️ Missing files: {missing} / {len(test_df)}")

# 6️⃣ Load images
test_images = []
valid_indices = []
for i, img_path in enumerate(test_df["image_file"]):
    if not os.path.exists(img_path):
        print("Missing:", img_path)
        continue
    img = load_img(img_path, target_size=(128, 128))
    arr = img_to_array(img) / 255.0
    test_images.append(arr)
    valid_indices.append(i)

test_images = np.array(test_images)
print(f"✅ Loaded {len(test_images)} valid test images")


Columns in DataFrame: Index(['id', 'text', 'img_path'], dtype='object')
                                                  id  \
0  d4c5e58ff5470b47be408208e16ce10ec9d49b663d8b3f...   
1  aad53da560d9a72f6b7559ac07a494b817b3b5aacb25a8...   
2  c1559459f7bd4003a7166d78c26c761a45429ec92e4176...   
3  51bc18d6d7cfa1b12088ddb9506156abad126f3c9c28d8...   
4  47928987d62780d8c0c4cce33e5f79346c75b6d4b805f2...   

                                                text  \
0  مرحبا وين صاربيت نتنياهو ؟\nاهلاء امشي لقدام ش...   
1                  بمنادسبة ان الامتحانات قربت\nدفوة   
2  {701011\nجرضب ضذ رذرابا\nاقعدى بميتين امك هيا ...   
3  هاثي بقي السؤال التالت\nDUERRR7 تاما درج\nايه ...   
4  ٥7 لما م ههو ام2 يسه مهيعمل المعء ماله 0صمه 8ب...   

                                            img_path  
0  ./data/images/facebook_images/islamic.stolen.m...  
1  ./data/images/instagram_images/data/archive/in...  
2  ./data/images/facebook_images/msr5er/msr5er/46...  
3  ./data/images/facebook_images/m

In [27]:
import zipfile


# Filter test_df for valid indices
test_df_valid = test_df.iloc[valid_indices].reset_index(drop=True)

# Prepare text embeddings
# (make sure you have defined get_bert_embeddings somewhere)
test_texts = list(test_df_valid["text"])
test_text_embeddings = get_bert_embeddings(test_texts, batch_size=32)
test_text_embeddings = np.array(test_text_embeddings)
print(f"✅ Created {len(test_text_embeddings)} text embeddings")

# Predict
# (make sure 'model' is already loaded)
test_preds = model.predict([test_images, test_text_embeddings], batch_size=32)
test_preds = np.argmax(test_preds, axis=1)

# Convert predictions to labels
label_map = {0: "not-hate", 1: "hate"}
predicted_labels = [label_map[p] for p in test_preds]

# Prepare submission
submission_df = pd.DataFrame({
    "id": test_df_valid["id"],
    "prediction": predicted_labels
})

# Save CSV and zip it
submission_df.to_csv("prediction.csv", index=False, encoding="utf-8")
with zipfile.ZipFile("prediction.zip", "w", zipfile.ZIP_DEFLATED) as zipf:
    zipf.write("prediction.csv")

print("✅ Done! prediction.zip is ready with", len(submission_df), "entries.")


from IPython.display import FileLink

# Create a clickable link to download prediction.zip
display(FileLink('prediction.zip'))


✅ Created 500 text embeddings
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step
✅ Done! prediction.zip is ready with 500 entries.


/kaggle/working/prediction.zip